# Point spread function using Tiny Tim/Galfit

Calling tinytim to generate psf is realized by .py script modified from Bryan R. Gillis, the following infomation is required:

0. sensor name

1. detector position

2. wavelengths of the emission lines (redshift)

3. subsampled factor

4. (optional) filter used and focus info



### import modules and defing a few useful little functions

In [1]:
import psf.make_psf as tinytim
import  numpy               as     np
from    astropy.table       import Table
from    astropy.io          import fits
from    scipy.ndimage       import rotate
#from    astropy.cosmology   import Planck18
#import  astropy.units       as     u
import  matplotlib.pyplot   as     plt
import  matplotlib.colors   as     colors  
from    matplotlib          import use
from    tqdm                import tqdm
from concurrent.futures     import ThreadPoolExecutor, as_completed
import sys, os
from IPython.display import clear_output
from scripts.tools   import *

obj_lis = Table.read('obj_lis_selected.fits')

###  gen imput parameters for tinytim

In [2]:
def gen_psf_input_param(obj):
    #try:
    #colnames for param table
    col_names = ['identifier', 'rootname',  'pad', 'filter', 'pa' ,'focus', 'coord_ha', 'coord_hb', 'wavelen_ha', 'wavelen_hb','DELTATIM']
    beam_path = f"data_products/{file_name(obj,'beams')}"
    with fits.open(beam_path) as hdu:
        rows = []
        for image in hdu:
            if image.name == 'SCI':
                #rootname, pad, shape filter
                rootname = image.header['ROOTNAME']
                identifier = f"{obj['subfield']}_{obj['id']}_{image.header['ROOTNAME']}"
                pad        = image.header.get('PAD',0)
                shape      = np.array(image.data.shape)
                filter     = image.header['filter']

                #thumbnail rel. pos
                x0_crop = image.header['ORIGINX']
                y0_crop = image.header['ORIGINY']

                # this part need actual pixel coord for each Ha and Hb wavelen
                coord_ha =  np.array((x0_crop,y0_crop)) - pad  #shape/2 +
                coord_hb =  np.array((x0_crop,y0_crop)) - pad  #shape/2 

                # this still needs focus information
                focus = 0
                #positon angle
                pa = image.header['ORIENTAT']

                #generate spectrum for halpha and hb
                wavelen_ha = 656.28*(1+obj['z_MAP'])
                wavelen_hb = 486.13*(1+obj['z_MAP'])

                #integrationtime
                deltatim = image.header['DELTATIM']

                rows.append((identifier, rootname,  pad, filter, pa ,focus, coord_ha, coord_hb, wavelen_ha, wavelen_hb,deltatim))
        
        #save tiny tim & drizzle input param table
        Table(rows=rows,names=col_names).write(f"psf/obj_param/{obj['subfield']}_{obj['id']}_psf.fits",overwrite=True)
        return f"{obj['subfield']}_{obj['id']} processed"
        
    #except Exception as e:
    #    return f"!error {obj['subfield']}_{obj['id']}: {e}"


from concurrent.futures     import ThreadPoolExecutor, as_completed
def cat_process_gen_psf_input_param(obj_lis,max_threads=1):
    #make directories:
    os.makedirs('psf/obj_param/',exist_ok=True)
    results = []
    if max_threads>1:
        with ThreadPoolExecutor(max_threads) as executor:                                                       
            futures = {executor.submit(gen_psf_input_param,obj):obj for obj in obj_lis}
            for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                results.append(future.result())
        return results
    else:
        for obj in tqdm(obj_lis):
            results.append(gen_psf_input_param(obj))
        return results


if __name__ == '__main__':
    obj_lis = Table.read('obj_lis_selected.fits')
    results = cat_process_gen_psf_input_param(obj_lis,max_threads=8)
    number = 0
    for result in results:
        if 'error' in result:
            number +=1
            print(result)
    print('total number of obj processed:',len(results))
    print('number of failed obj',number)


Processing: 100%|██████████| 158/158 [01:30<00:00,  1.75it/s]

total number of obj processed: 158
number of failed obj 0


In [ ]:
pas = np.array([])
filters = np.array([])
num_pas_per_exposure = np.array([])
num_exposure_per_obj = np.array([])
for obj in tqdm(obj_lis): 
    table  = Table.read(f"psf/obj_param/{obj['subfield']}_{obj['id']}_psf.fits")
    pas    = np.append(pas,table['pa'])
    filters = np.append(filters,table['filter'])
    num_pas_per_exposure = np.append(num_pas_per_exposure,len(np.unique(np.round(table['pa'],decimals=1))))
    num_exposure_per_obj = np.append(num_exposure_per_obj,len(table))



100%|██████████| 158/158 [00:01<00:00, 117.44it/s]


In [67]:
print('number of exposure per obj:',np.mean(num_exposure_per_obj))
print('number of pas per exposure:',np.mean(num_pas_per_exposure))
print('number of unique pas:',len(np.unique(np.round(pas,1))))
print('number of exposures per pa:')
g102 = filters==b'G102'
g141 = filters==b'G141'


print('average number of exposures in 102 grism',np.mean(np.unique(np.round(pas,1)[g102],return_counts=True)[1]))
print('average number of exposures in 141 grism',np.mean(np.unique(np.round(pas,1)[g141],return_counts=True)[1]))

number of exposure per obj: 44.4873417721519
number of pas per exposure: 4.525316455696203
number of unique pas: 65
number of exposures per pa:
average number of exposures in 102 grism 121.53191489361703
average number of exposures in 141 grism 65.85


### generate psf by calling tinytim + rotation and crop

In [3]:
def rotate_and_crop(image_path,rotating_angle=0,crop_size=30):
    with fits.open(image_path) as hdu:

        rot = rotate(hdu[0].data, 
                    angle=rotating_angle, 
                    reshape=False, 
                    mode='constant')
        
        center_x, center_y = np.array(hdu[0].data.shape) // 2
        half_crop = crop_size // 2
        cropped_data = rot[
            center_y - half_crop : center_y + half_crop,
            center_x - half_crop : center_x + half_crop]
        image = fits.ImageHDU(data=cropped_data, header=hdu[0].header)
    return image

#using tinytim and rotate to generate single psf
def gen_psf(obj,exist_skip=False):
    #try:

    #path to save individual psf and combined psf
    save_path          = f"psf/individual_psf/{obj['subfield']}_{obj['id']}"
    save_path_combined = f"psf/combined_psf/{obj['subfield']}_{obj['id']}"

    if os.path.exists(save_path) and exist_skip==True: 
        clear_output(wait=True)
        return f"{obj['subfield']}_{obj['id']} skipped"

    #load parameter table
    param_table = Table.read(f"psf/obj_param/{obj['subfield']}_{obj['id']}_psf.fits")
    #use integration time as weight
    int_time = np.array(param_table['DELTATIM'])
    weights = int_time/np.sum(int_time)
    #psf_lis to contain individual psf
    psf_lis = fits.HDUList(); psf_lis.append(fits.PrimaryHDU())
    #_combine to store weighted average
    ha_combine = fits.ImageHDU(data=np.zeros((30,30)));hb_combine = fits.ImageHDU(data=np.zeros((30,30)))
    
    #calculate for each individual beam file:
    for i,row in tqdm(enumerate(param_table)):
        print(row)
        identifier, rootname, pad, filter, pa ,focus, coord_ha, coord_hb, wavelen_ha, wavelen_hb, deltatim = row

#------------------this part generate and rotate individual psf--------------------------------
        #ha
        filename_ha = f"psf/individual_psf/{identifier.split('_')[2]}_ha.fits"
        tinytim.make_subsampled_model_psf(filename=filename_ha,
                            psf_size = 4,
                            filter_name = filter,
                            focus = focus,  
                            psf_position = coord_ha,
                            mono = wavelen_ha,
                            subsampling_factor = 2,
                            exist_skip=True)
        ha_psf = rotate_and_crop(filename_ha,-pa)
        ha_psf.name= f'{rootname}_ha'
        ha_combine.data += ha_psf.data * weights[i]
        
        #hb
        filename_hb = f"psf/individual_psf/{identifier.split('_')[2]}_hb.fits"
        tinytim.make_subsampled_model_psf(filename=filename_hb,
                            psf_size = 4,
                            filter_name = filter,
                            focus = focus,  
                            psf_position = coord_hb,
                            mono = wavelen_hb,
                            subsampling_factor = 2,
                            exist_skip=True)
        hb_psf = rotate_and_crop(filename_hb,-pa)
        hb_psf.name = f'{rootname}_hb'
        hb_combine.data += hb_psf.data * weights[i]
#----------------------------------------------------------------------------------------------

        #normalization and append individual psf
        ha_psf.data = ha_psf.data/np.sum(ha_psf.data)
        hb_psf.data = hb_psf.data/np.sum(hb_psf.data)
        psf_lis.append(ha_psf);psf_lis.append(hb_psf)
        os.remove(filename_ha);os.remove(filename_hb)

    #save combined & individual psf
    psf_lis.append(ha_combine);psf_lis.append(hb_combine)
    psf_lis.writeto(f'{save_path}_psf.fits',overwrite=True)
    ha_combine.writeto(f'{save_path_combined}_ha.fits',overwrite=True)
    hb_combine.writeto(f'{save_path_combined}_hb.fits',overwrite=True)

    #clear output and return
    clear_output(wait=True)
    return f"{obj['subfield']}_{obj['id']} processed"

    #except Exception as e:
    #    return f"!error {obj['subfield']}_{obj['id']}:{e}"



from concurrent.futures     import ThreadPoolExecutor, as_completed
def cat_process_gen_psf_from_input_param(obj_lis,max_threads=1):
    #make directories:
    os.makedirs('psf/individual_psf',exist_ok=True)
    os.makedirs('psf/combined_psf',exist_ok=True)  
    results = []
    if max_threads>1:
        with ThreadPoolExecutor(max_threads) as executor:
            futures = {executor.submit(gen_psf,obj):obj for obj in obj_lis}
            for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                results.append(future.result())
        return results
    else:
        for obj in tqdm(obj_lis):
            results.append(gen_psf(obj))
        return results


if __name__ == '__main__':
    obj_lis = Table.read('obj_lis_selected.fits')
    results = cat_process_gen_psf_from_input_param(obj_lis,max_threads=1)
    number = 0
    for result in results:
        if 'error' in result:
            number +=1
            print(result)
    print('total number of obj processed:',len(results))
    print('number of failed obj',number)

  1%|▏         | 2/158 [00:05<07:32,  2.90s/it]

     identifier      rootname pad filter         pa        focus  coord_ha   coord_hb      wavelen_ha         wavelen_hb      DELTATIM 
------------------- --------- --- ------ ----------------- ----- ---------- ---------- ------------------ ------------------ ----------
GN7_11839_icat07bxq icat07bxq 200   G102 19.63040112387118     0 365 .. -19 365 .. -19 1470.0944149636491 1088.9513590941044 100.000313


     identifier      rootname pad filter         pa        focus  coord_ha   coord_hb      wavelen_ha         wavelen_hb      DELTATIM 
------------------- --------- --- ------ ----------------- ----- ---------- ---------- ------------------ ------------------ ----------
GN7_11839_icat07c2q icat07c2q 200   G102 19.63098091015885     0 376 .. -15 376 .. -15 1470.0944149636491 1088.9513590941044 100.000313


     identifier      rootname pad filter         pa        focus  coord_ha  coord_hb     wavelen_ha         wavelen_hb      DELTATIM 
------------------- --------- --- ------ ----------------- ----- --------- --------- ------------------ ------------------ ----------
GN7_11839_icat07c9q icat07c9q 200   G102 19.63058503786886     0 372 .. -8 372 .. -8 1470.0944149636491 1088.9513590941044 100.000313


     identifier      rootname pad filter         pa        focus  coord_ha   coord_hb      wavelen_ha         wavelen_hb      DELTATIM 
------------------- --------- --- ------ ----------------- ----- ---------- ---------- ------------------ ------------------ ----------
GN7_11839_icat07ceq icat07ceq 200   G102 19.62997320548297     0 362 .. -12 362 .. -12 1470.0944149636491 1088.9513590941044 100.000313


...

     identifier      rootname pad filter        pa        focus coord_ha coord_hb     wavelen_ha         wavelen_hb      DELTATIM 
------------------- --------- --- ------ ---------------- ----- -------- -------- ------------------ ------------------ ----------
GN7_11839_icxt39esq icxt39esq 200   G102 54.6232319886229     0 43 .. 66 43 .. 66 1470.0944149636491 1088.9513590941044 100.000313


     identifier      rootname pad filter         pa        focus coord_ha coord_hb     wavelen_ha         wavelen_hb      DELTATIM 
------------------- --------- --- ------ ----------------- ----- -------- -------- ------------------ ------------------ ----------
GN7_11839_icxt39euq icxt39euq 200   G102 54.62298969655579     0 33 .. 62 33 .. 62 1470.0944149636491 1088.9513590941044 100.000313


8it [00:03,  2.61it/s]
  1%|▏         | 2/158 [00:08<11:29,  4.42s/it]


     identifier      rootname pad filter         pa        focus coord_ha coord_hb     wavelen_ha         wavelen_hb      DELTATIM 
------------------- --------- --- ------ ----------------- ----- -------- -------- ------------------ ------------------ ----------
GN7_11839_icxt40npq icxt40npq 200   G102 54.62318017561582     0 37 .. 55 37 .. 55 1470.0944149636491 1088.9513590941044 100.000313


KeyboardInterrupt: 